In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pytz
import numpy as np

In [ ]:
data_posts_hashtags_raw = pd.read_csv("../data/posts_hashtags_as_array.csv")
data_posts_hashtags = data_posts_hashtags_raw.drop(columns = ['id'])
data_posts_hashtags['text'][0]

12271 rows

**Data preperation**

In [ ]:
# Count NaNs per column
nan_counts = data_posts_hashtags.isna().sum()

# Count unique values per column
unique_counts = data_posts_hashtags.nunique()

# Combine into one table
summary = pd.DataFrame({
    'NaNs': nan_counts,
    'Unique Values': unique_counts
})

print(summary)

isMuted, author_region, location_address, author_ttSeller have a lot of NaNs. author_region is always empty


In [ ]:
#Drop author region
data_posts_hashtags = data_posts_hashtags_raw.drop(columns = ['author_region'])

In [ ]:
data_posts_hashtags["createTimeISO"] = pd.to_datetime(data_posts_hashtags["createTimeISO"])
data_posts_hashtags["country"] = data_posts_hashtags["location_address"].str.extract(r',\s*([^,]+)$')


**Language distribution**

In [ ]:
# Count languages
lang_counts = data_posts_hashtags['textLanguage'].value_counts()

# Convert to DataFrame and calculate percentage
lang_dist = lang_counts.reset_index()
lang_dist.columns = ['Language', 'Count']
lang_dist['Percentage'] = (lang_dist['Count'] / lang_dist['Count'].sum()) * 100

# Sort by most common
lang_dist = lang_dist.sort_values(by='Count', ascending=False).reset_index(drop=True)

# Display top 5
lang_dist.head(5)

75% of all posts are in German in English therefore we will from know on limit our analysis to those two languages

In [ ]:
data_english = data_posts_hashtags[data_posts_hashtags.textLanguage.isin(['en'])]
data_german = data_posts_hashtags[data_posts_hashtags.textLanguage.isin(['de'])]

**Content with advertisements**

In [ ]:
cols = ["isAd", "isSponsored"]

plt.figure(figsize=(10, 4))
for i, col in enumerate(cols, 1):
    plt.subplot(1, 2, i)
    sns.countplot(x=col, data=data_posts_hashtags)
    plt.title(col)

plt.tight_layout()
plt.show()

In [ ]:
(data_posts_hashtags["isAd"] == data_posts_hashtags["isSponsored"]).sum()

(data_posts_hashtags["isAd"] != data_posts_hashtags["isSponsored"]).sum()

Nearly all of the data are not ads nor sponsored. 582 rows is_ad and is_sponsred is different. This makes only 5.5%. Meaning for 95% of the data is_ad equals to is_sponsored

**Success metrics**

In [ ]:
metrics = ["playCount", "diggCount", "commentCount", "collectCount", "shareCount"]

In [ ]:
# Create a melted DataFrame with a new column "is_zero"
metrics_is_zero = data_posts_hashtags[metrics].melt(var_name="metric", value_name="value")
metrics_is_zero["is_zero"] = metrics_is_zero["value"] == 0

plt.figure(figsize=(12, 6))
sns.countplot(data=metrics_is_zero, x="metric", hue="is_zero")
plt.title("Share of post with 0 play_counts, comments etc")
plt.legend(title="Is Zero")
plt.tight_layout()
plt.show()

In [ ]:
(data_posts_hashtags["playCount"] == 0).sum()

We can see that there is a quite high share of posts which have no comments etc

In [ ]:
plt.figure(figsize=(20, 20))
for i, col in enumerate(metrics, 1):
    plt.subplot(3, 3, i)
    q99 = data_posts_hashtags[col].quantile(0.99)
    filtered = data_posts_hashtags[(data_posts_hashtags[col] > 0) & (data_posts_hashtags[col] < q99)]
    sns.histplot(filtered[col], bins=50, kde=True)
    plt.title(f"{col} (<99th percentile)")

plt.tight_layout()
plt.show()

The success metrics are gighly skweded 

In [ ]:
for col in metrics:
    q99 = data_posts_hashtags[col].quantile(0.99)
    max_val = data_posts_hashtags[col].max()
    print(f"{col}: 99th perc.:{q99:.0f}, max: {max_val}")

**Video duration**

In [ ]:
sns.boxplot(x=data_posts_hashtags["video_duration"])
plt.title("Video Duration Spread")
plt.xlabel("Video Duration (seconds)")
plt.show()

In [ ]:
filtered = data_posts_hashtags[data_posts_hashtags["video_duration"] <= 120]
sns.histplot(filtered["video_duration"], bins=50, kde=True)
plt.title("Distribution of Video Durations ≤ 120s")
plt.xlabel("Video Duration (seconds)")
plt.ylabel("Count")
plt.show()

**From where are our posts**

In [ ]:
data_german["country"].unique()

In [ ]:
data_english["country"].unique()

In [ ]:
data_posts_hashtags["country"].unique()

In [ ]:
country_fix = {
    "Deutschland": "Germany",
    "Österreich": "Austria",
    "UK": "United Kingdom",
    "USA": "United States",
    "Brasil": "Brazil",
    "Italia": "Italy",
    "مصر": "Egypt",
    "المملكة العربية السعودية": "Saudi Arabia",
    "Turkey": "Türkiye",
    "Bangkok 10300 Thailand": "Thailand",
    "Mandarin Oriental Hotel Mandarin Oriental": None,  # unclear
}

# Apply the fixes
data_posts_hashtags["country_clean"] = data_posts_hashtags["country"].replace(country_fix)

In [ ]:
timezone_map = {
    "Germany": "Europe/Berlin",
    "France": "Europe/Paris",
    "Türkiye": "Europe/Istanbul",
    "Saudi Arabia": "Asia/Riyadh",
    "United States": "America/New_York",  # default — or map by state later
    "Canada": "America/Toronto",
    "Switzerland": "Europe/Zurich",
    "South Korea": "Asia/Seoul",
    "Pakistan": "Asia/Karachi",
    "Indonesia": "Asia/Jakarta",
    "Greece": "Europe/Athens",
    "Philippines": "Asia/Manila",
    "Sweden": "Europe/Stockholm",
    "Kenya": "Africa/Nairobi",
    "Egypt": "Africa/Cairo",
    "South Africa": "Africa/Johannesburg",
    "Austria": "Europe/Vienna",
    "United Kingdom": "Europe/London",
    "Nigeria": "Africa/Lagos",
    "Poland": "Europe/Warsaw",
    "Bangladesh": "Asia/Dhaka",
    "Australia": "Australia/Sydney",
    "Brazil": "America/Sao_Paulo",
    "Colombia": "America/Bogota",
    "United Arab Emirates": "Asia/Dubai",
    "Italy": "Europe/Rome",
    "Czechia": "Europe/Prague",
    "Thailand": "Asia/Bangkok",
    "Mexico": "America/Mexico_City",
    "Nederland": "Europe/Amsterdam",
    "Japan": "Asia/Tokyo",
    "Algeria": "Africa/Algiers"
}


In [ ]:
# Convert createTimeISO to UTC datetime
data_posts_hashtags['utc_time'] = pd.to_datetime(data_posts_hashtags['createTimeISO'], utc=True)

# Nur für gültige Länder die lokale Zeit berechnen
data_posts_hashtags['local_time'] = [
    utc.astimezone(pytz.timezone(timezone_map[country])) if pd.notna(country) and country in timezone_map else pd.NaT
    for utc, country in zip(data_posts_hashtags['utc_time'], data_posts_hashtags['country'])
]

# Stunde extrahieren
data_posts_hashtags['hour'] = data_posts_hashtags['local_time'].astype(str).str.extract(r'(\d{2}):')[0].astype('Int64')


**From when are our posts**

In [ ]:
mask = data_posts_hashtags["createTimeISO"].dt.date >= pd.to_datetime("2024-10-01").date()
posts_per_day = data_posts_hashtags.loc[mask, "createTimeISO"].dt.date.value_counts().sort_index()

plt.figure(figsize=(12, 5))
posts_per_day.plot()
plt.title("Number of Posts Per Day")
plt.xlabel("Date")
plt.ylabel("Post Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
posts_per_day = data_posts_hashtags["createTimeISO"].dt.date.value_counts().sort_index()

plt.figure(figsize=(12, 5))
posts_per_day.plot()
plt.title("Number of Posts Per Day")
plt.xlabel("Date")
plt.ylabel("Post Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Are posts from before 2025 even relevant for us?

In [ ]:
sns.countplot(x="hour", data=data_posts_hashtags)
plt.title("Posts by Hour of Day")
plt.xlabel("Hour")
plt.ylabel("Post Count")
plt.show()

People post most often in the evening between 20:00 to 22:00. During the night there are the least posts.

Need to double check that this is correct!

In [ ]:
data_posts_hashtags["weekday"] = data_posts_hashtags["createTimeISO"].dt.day_name()

sns.countplot(x="weekday", data=data_posts_hashtags,
              order=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
plt.title("Posts by Weekday")
plt.xlabel("Day")
plt.ylabel("Post Count")
plt.xticks(rotation=45)
plt.show()


**Influencer analysis**

In [ ]:
bins = [0, 1_000, 10_000, 100_000,500_000, 1_000_000, np.inf]
labels = ['<1k', '1k–10k', '10k–100k', '100k–500k', '500k–1M', '1M+']
data_posts_hashtags['fans_bin'] = pd.cut(data_posts_hashtags['author_fans'], bins=bins, labels=labels)

sns.countplot(x='fans_bin', data=data_posts_hashtags)
plt.title("Authors Grouped by Follower Count")
plt.xlabel("Follower Range")
plt.ylabel("Count")
plt.show()

### Hashtags Exploration

**Hashtags that were searched for**

In [ ]:
unique_searchhashtags = data_posts_hashtags['searchHashtag_name'].unique() 

search_hashtag_counts = data_posts_hashtags.searchHashtag_name.value_counts()

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(x=search_hashtag_counts.values, y=search_hashtag_counts.index, palette="pastel")
plt.title("Search Hashtags")
plt.xlabel("Count")
plt.ylabel("Hashtag")
plt.show()

**Most common and engaging hashtags**

Sidenote: Looking in Bigquery found out that search_hashtag is actually not always part of the hashtags

In [ ]:
data_hashtags = pd.read_csv("../data/posts_hashtags_selected_columns.csv")
data_hashtags = data_hashtags.drop(columns = ['bucketUrl'])
data_hashtags

In [ ]:
(data_hashtags['searchHashtag_name'] == data_hashtags['hashtag_name']).sum()

In [ ]:
unique_hashtags = data_hashtags['hashtag_name'].unique() #4545 unique hashtags

hashtag_counts = data_hashtags.hashtag_name.value_counts().head(30)  # top 30

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(x=hashtag_counts.values, y=hashtag_counts.index, palette="pastel")
plt.title("Top 30 Hashtags")
plt.xlabel("Count")
plt.ylabel("Hashtag")
plt.tight_layout()
plt.show()

In [ ]:
hashtag_post_counts = (
    data_hashtags.groupby('hashtag_name')['id']
    .nunique()
    .sort_values(ascending=False)
)

quantiles = hashtag_post_counts.quantile([0.25, 0.5, 0.75, 0.9, 0.95, 0.975, 0.99])
print("Quantile der Postanzahl pro Hashtag:")
print(quantiles)

In [ ]:
#Let's look at the number of views for each Hashtag

# number of hashtags per post
data_hashtags['hashtag_count'] = data_hashtags.groupby('id')['hashtag_name'].transform('count')

# Distribute metric
for metric in ['playCount', 'diggCount', 'shareCount', 'commentCount']:
    data_hashtags[f'{metric}_per_hashtag'] = data_hashtags[metric] / data_hashtags['hashtag_count']

def plot_sum_and_avg_by_metric(data):
    metrics = ['playCount', 'diggCount', 'shareCount', 'commentCount']
    
    # Calculate fair share per hashtag
    data['hashtag_count'] = data.groupby('id')['hashtag_name'].transform('count')
    for metric in metrics:
        data[f'{metric}_per_hashtag'] = data[metric] / data['hashtag_count']
    
    for metric in metrics:
        metric_col = f'{metric}_per_hashtag'
        
        agg = (
            data.groupby('hashtag_name')
            .agg(
                post_count=('id', 'nunique'),
                total_metric=(metric_col, 'sum')
            )
            .assign(avg_metric_per_post=lambda x: x['total_metric'] / x['post_count'])
        )

        # Filter: only hashtags with at least 30 posts
        agg = agg[agg['post_count'] >= 20]

        # Top 30 by total and by average
        top_total = agg.sort_values('total_metric', ascending=False).head(30)
        top_avg = agg.sort_values('avg_metric_per_post', ascending=False).head(30)

        # Plot
        fig, axes = plt.subplots(1, 2, figsize=(22, 10))

        sns.barplot(
            x=top_total['total_metric'],
            y=top_total.index,
            hue=top_total.index,
            palette='pastel',
            legend=False,
            ax=axes[0]
        )
        axes[0].set_title(f"Top 30 Hashtags by Total {metric}")
        axes[0].set_xlabel("Estimated Total (split across hashtags)")

        sns.barplot(
            x=top_avg['avg_metric_per_post'],
            y=top_avg.index,
            hue=top_avg.index,
            palette='pastel',
            legend=False,
            ax=axes[1]
        )
        axes[1].set_title(f"Top 30 Hashtags by Avg {metric} per Post")
        axes[1].set_xlabel("Average per Post (adjusted)")

        plt.tight_layout()
        plt.show()


plot_sum_and_avg_by_metric(data_hashtags)

In [5]:
import pandas as pd
df = pd.read_csv('../data/posts_transcripts.csv')

In [6]:
df

,post_id,text,textLanguage,createTimeISO,isAd,author_nickName,author_signature,author_fans,video_duration,diggCount,...,collectCount,commentCount,isSponsored,searchHashtag_name,searchHashtag_views,location_address,detected_language,transcribed_text,video_description,date
0,7472533937667968278,Aquafacial🫧 Boost für deine Haut💆🏻‍♀️ Tief ger...,de,2025-02-17 23:28:17+00:00,False,SKINID_STUDIO,all about ✨Aquafacial & Microneedling✨ \nat @k...,32,10,17,...,2,2,False,hautpflege,2700000000,"Nurnberg, Bavaria, Germany",NaN,NaN,Here’s a close-up clip of a woman getting a fa...,2025-02-17
1,7472855983593721090,Hautpflege und Enzym–Peeling bei empfindlicher...,de,2025-02-18 20:17:53+00:00,False,Sabi_шка🩵🤹‍♂️,🍃Bio-Nature-Beauty-Skincare\nGermanistik/Lingu...,10300,137,21,...,5,10,False,hautpflege,2700000000,"Nordrhein-Westfalen, Deutschland",German,"Hey, meine lieben Mädels, ich leide seit meine...",A beauty influencer shares her secret to perfe...,2025-02-18
2,7472768498637327638,SkinGlow - Aqua Facial☁️✨ Instagram: Asli.Beau...,en,2025-02-18 14:38:26+00:00,False,𝐀𝐬𝐥𝐢 🪬,𝐊𝐃𝐇 | 𝐍𝐢𝐂𝐨🦁𝟏𝐱🥈𝟓𝐱🥉\n𝐈𝐆:𝐀𝐬𝐥𝐢.𝐁𝐞𝐚𝐮𝐭𝐞𝐞📍𝐒𝐭𝐮𝐭𝐭𝐠𝐚𝐫𝐭,13700,9,379,...,2,5,False,hautpflege,2700000000,"Stuttgart, Baden Wurttemberg, Germany",English,I got so tight all over her body. She just wan...,"Watch this lady get an Aqua Facial, and see ho...",2025-02-18
3,7472518128191212822,Verjüngung ohne Chirurgie ist möglich. 💉 Mit B...,de,2025-02-17 22:26:52+00:00,False,MKG Rothenbaum,MKG Rothenbaum J.C.Castillo Duque\nFacharzt fü...,228,12,6,...,2,1,False,hautpflege,2700000000,"Rothenbaumchaussee 5, 20148 Hamburg, Germany",English,Game start. Appart apartment. Appart apartment...,A man drinks from a mug as text appears on the...,2025-02-17
4,7472508255441276182,☞𝕎𝕖𝕣𝕓𝕦𝕟𝕘☜ 🔻Produktvorstellung🔻 ᗰᗩᒪ Gᗩᑎᘔ EᕼᖇᒪIᑕ...,de,2025-02-17 21:48:32+00:00,False,Nicki.sally.testet,NaN,1145,36,26,...,4,4,False,hautpflege,2700000000,"Mulheim, North Rhine Westphalia, Germany",German,"Mal ganz ehrlich, wer hat keinen Schrank bei s...",This video promotes two Lancome products for a...,2025-02-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6506,7472467511544466710,Ihr habt keinen Haarausfall ok 😭 Bitte tik tok...,de,2025-02-17 19:10:24+00:00,False,xskincare | Leon,Kosmetik Entwickler \nHautpflege auf wissensch...,945300,64,102000,...,7302,947,False,hautpflege,3100000000,"Berlin, Berlin, Germany",German,"falls euch das betrifft, ist dieses Video für ...",A creator is reacting to a video of hair loss....,2025-02-17
6507,7517319980791172374,Wie aus Stahl Präzision wird – die Kunst der p...,de,2025-06-18 16:00:54+00:00,False,Syber Secrets,✨Faktenliebhaber & Geschichtenerzähler.📚\n🔍 En...,16900,62,3114,...,381,15,False,hautpflege,3100000000,"Berlin, Berlin, Germany",German,Wenn du dich mit einem herkömmlichen Rasierer ...,"In diesem Video wird erklärt, wie sich die Ras...",2025-06-18
6508,7519028281006378262,Verwöhne deine Haut mit der Kraft der Schnecke...,de,2025-06-23 06:30:06+00:00,False,Tiyad DE3,zIUj,2505,30,3,...,0,0,True,hautpflegeroutine,267400000,"Berlin, Berlin, Germany",NaN,NaN,NaN,2025-06-23
6509,7517847677955034390,"Wer braucht Filter, wenn deine Haut von allein...",de,2025-06-20 02:09:03+00:00,True,Tiyad DE3,zIUj,2506,30,6,...,0,1,True,hautpflegeroutine,266000000,"Berlin, Berlin, Germany",NaN,NaN,NaN,2025-06-20
